<a href="https://colab.research.google.com/github/AKMADOU/Assigment_NLP_ADOU_Mathurin/blob/main/ADOU_Kouam%C3%A9_Mathurin_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this lab is to implement a language identifier (LID).

Our first model will be based on Naive Bayes.

In [14]:
import io, sys, math, re
import numpy as np
from collections import defaultdict

The next function is used to load the data. Each line of the data consist of a label (corresponding to a language), followed by some text, written in that language. Here is an example of data:

```__label__de Zur Namensdeutung gibt es mehrere Varianten.```


In [2]:
def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

You can now try loading the first dataset `train1.txt` and look what examples look like.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt")
print(data[0])

('__label__de', ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.'])


Next, we will start implementing the Naive Bayes method. This technique is based on word counts, and we thus need to start by implementing a function to count the words and labels of our training set.

`n_examples` is the total number of examples

`n_words_per_label` is the total number of words for a given label

`label_counts` is the number of times a given label appears in the training data

`word_counts` is the number of times a word appears with a given label

In [18]:
def count_words(data):
    n_examples = 0
    n_words_per_label = defaultdict(lambda: 0)
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for label, sentence in data:
        for word in sentence:
          word_counts[label][word]+=1
        label_counts[label]+=1
        n_examples=n_examples+1
        n_words_per_label[label]=len(sentence)    
            

    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'n_examples': n_examples, 
            'n_words_per_label': n_words_per_label}

In [ ]:
count_words(data)

Next, using the word and label counts from the previous function, we can implement the prediction function.

Here, `mu` is a regularization parameter (Laplace smoothing), and `sentence` is the list of words corresponding to the test example.

In [15]:
def predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label):
    best_label = None
    best_score = float('-inf')
    
    for label in word_counts.keys():
        score = 0.0
        for word in sentence:
          b=mu*len(word_counts[label])+n_words_per_label[word]
          a=mu+word_counts[label][word]
          score+=np.log(a/b)

          if score>best_score:
            best_score=score 
            best_label=label  
    return best_label

The next function will be used to evaluate the Naive Bayes model on a validation set. It computes the accuracy for a particular regularization parameter `mu`.

In [16]:
def compute_accuracy(valid_data, mu, counts):
    accuracy = 0.0
    for label, sentence in valid_data:
      y_pred=predict(sentence, mu, counts["label_counts"],counts["word_counts"], counts["n_examples"], counts["n_words_per_label"])
      if y_pred==label:
        accuracy =accuracy+1
    return (accuracy/len(valid_data))*100

In [17]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt")
valid_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/valid1.txt")
counts = count_words(train_data)
compute_accuracy(valid_data, mu, counts)
print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")


** Naive Bayes **

Validation accuracy: 72.900

